In [12]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

In [13]:
season = "MONSOON"
model_folder = "random_forest" 
model_name = "Random_Forest_t0.410"

# Configure season window and output directories
if season.upper() == "PREMONSOON":
    start_month, end_month = 3,5   # Mar–May
else:
    start_month, end_month = 6, 7  # Jun–Oct (adjust if needed)

CONFIG = {
    "season": season,
    "start_month": start_month,
    "end_month": end_month,
    # Observations file (relative to workspace, matching project structure)
    "obs_excel": Path("..") / "LSTM" / "PREMONSOON-janjul201724.xlsx",
    # Base dir where prediction CSVs and outputs live
    "base_dir": Path("predictionsv2") / model_folder / model_name / season,
    # Explicit predictions dir alias for convenience
    "predictions_dir":  Path("predictionsv2") / model_folder / model_name / season,
}

print(f"CONFIG: season={CONFIG['season']}, window months={CONFIG['start_month']}-{CONFIG['end_month']}")

CONFIG: season=MONSOON, window months=6-7


In [14]:
# Load event flags across years and build a single dataframe indexed by Date
years = [2017, 2022, 2023, 2024]
base_dir = CONFIG['base_dir']

frames = []
for y in years:
    csv_path = base_dir / f"sum_f_{y}.csv"
    df_y = pd.read_csv(csv_path)

    # Normalize date column
    date_col = None
    for cand in ["Date", "date", "DATE", "Date Time", "datetime"]:
        if cand in df_y.columns:
            date_col = cand
            break
    if date_col is None:
        # If first column looks like date, use it
        first_col = df_y.columns[0]
        try:
            pd.to_datetime(df_y[first_col])
            date_col = first_col
        except Exception:
            raise ValueError(f"No date-like column found in {csv_path}")

    df_y["Date"] = pd.to_datetime(df_y[date_col]).dt.normalize()
    df_y = df_y.drop(columns=[c for c in [date_col] if c != "Date" and c in df_y.columns])
    frames.append(df_y)

sum_f = pd.concat(frames, ignore_index=True)
sum_f = sum_f.sort_values("Date").reset_index(drop=True)
sum_f_indexed = sum_f.set_index("Date")

print(f"Loaded event flags data: {len(sum_f_indexed)} rows, columns={list(sum_f_indexed.columns)}")

Loaded event flags data: 244 rows, columns=['1-day', '2-day', '3-day', '4-day', '5-day', '6-day', '7-day', '8-day', '9-day', '10-day']


In [15]:
# ====================================
# EVENT-BASED EVALUATION METRICS (All Years)
# ====================================

from sklearn.metrics import confusion_matrix

df_obs = pd.read_excel(CONFIG['obs_excel'])
df_obs['Date'] = pd.to_datetime(df_obs['Date Time']).dt.normalize()

# Calculate daily water levels and set threshold
wl_daily = df_obs.groupby('Date')['WL (mMSL)'].mean().reset_index()
threshold = 12.34 if CONFIG['season'].upper() == 'MONSOON' else 10.69
wl_daily['label'] = (wl_daily['WL (mMSL)'] >= threshold).astype(int)

# Allowed months for the configured season
allowed_months = list(range(CONFIG['start_month'], CONFIG['end_month'] + 1))

# Actual flood dates across all years within allowed months
flood_in_season = df_obs[(df_obs['WL (mMSL)'] >= threshold) &
                        (df_obs['Date'].dt.month.isin(allowed_months))]
actual_flood_dates = set(flood_in_season['Date'].dt.date.unique())

# Evaluation dates: all dates present in predictions within allowed months
all_dates = sorted([d for d in sum_f_indexed.index if d.month in allowed_months])

if len(all_dates) == 0:
    raise ValueError("No dates found in sum_f_indexed for the configured season months.")

# -----------------------------
# Lead-day wise metrics
# -----------------------------
results = []

for lead_col in sum_f_indexed.columns:
    # Skip non-lead columns if any
    if not isinstance(lead_col, str) or '-' not in lead_col:
        continue

    days = int(lead_col.split('-')[0])

    # Event flags: any non-zero means event (1), else 0
    flagged_idx = sum_f_indexed.index[sum_f_indexed[lead_col] != 0]

    # Verification dates (init_date + lead_days) within season months
    time_event = (flagged_idx + pd.Timedelta(days=days)).normalize()
    predicted_event_dates = {d.date() for d in time_event if d.month in allowed_months}

    y_true = np.array([d.date() in actual_flood_dates for d in all_dates]).astype(int)
    y_pred = np.array([d.date() in predicted_event_dates for d in all_dates]).astype(int)

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    TP, FN = cm[0, 0], cm[0, 1]
    FP, TN = cm[1, 0], cm[1, 1]

    precision = TP / (TP + FP) if (TP + FP) > 0 else float('nan')
    recall = TP / (TP + FN) if (TP + FN) > 0 else float('nan')
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 and not np.isnan(precision) and not np.isnan(recall) else float('nan')
    far = FP / (TP + FP) if (TP + FP) > 0 else float('nan')

    results.append({
        'Lead Day': lead_col,
        'TP (Hits)': TP,
        'FN (Misses)': FN,
        'FP (False Alarms)': FP,
        'TN (Correct Negatives)': TN,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'FAR': far,
    })

summary_df = pd.DataFrame(results)

# -----------------------------
# Overall metrics (union across all leads)
# -----------------------------
union_predicted = set()
for lead_col in sum_f_indexed.columns:
    if not isinstance(lead_col, str) or '-' not in lead_col:
        continue
    days = int(lead_col.split('-')[0])
    flagged_idx = sum_f_indexed.index[sum_f_indexed[lead_col] != 0]
    time_event = (flagged_idx + pd.Timedelta(days=days)).normalize()
    union_predicted.update({d.date() for d in time_event if d.month in allowed_months})

y_true_overall = np.array([d.date() in actual_flood_dates for d in all_dates]).astype(int)
y_pred_overall = np.array([d.date() in union_predicted for d in all_dates]).astype(int)

cm_o = confusion_matrix(y_true_overall, y_pred_overall, labels=[1, 0])
TP_o, FN_o = cm_o[0, 0], cm_o[0, 1]
FP_o, TN_o = cm_o[1, 0], cm_o[1, 1]

precision_o = TP_o / (TP_o + FP_o) if (TP_o + FP_o) > 0 else float('nan')
recall_o = TP_o / (TP_o + FN_o) if (TP_o + FN_o) > 0 else float('nan')
f1_o = 2 * precision_o * recall_o / (precision_o + recall_o) if (precision_o + recall_o) > 0 and not np.isnan(precision_o) and not np.isnan(recall_o) else float('nan')
far_o = FP_o / (TP_o + FP_o) if (TP_o + FP_o) > 0 else float('nan')

overall_row = pd.DataFrame([{ 
    'Lead Day': 'Overall (union)',
    'TP (Hits)': TP_o,
    'FN (Misses)': FN_o,
    'FP (False Alarms)': FP_o,
    'TN (Correct Negatives)': TN_o,
    'Precision': precision_o,
    'Recall': recall_o,
    'F1 Score': f1_o,
    'FAR': far_o,
}])

summary_with_overall = pd.concat([summary_df, overall_row], ignore_index=True)

print("\n" + "="*100)
print("EVENT-BASED EVALUATION METRICS (All Years) BY LEAD DAY + OVERALL")
print("="*100)
print(summary_with_overall.to_string(index=False))

# Save summary to CSV (same directory as predictions)
out_summary = CONFIG['base_dir'] / 'evaluation_metrics_all_years.csv'
summary_with_overall.to_csv(out_summary, index=False)
print(f"\nSaved evaluation metrics to: {out_summary}")

# -----------------------------
# Detailed dates by lead (padded)
# -----------------------------
dates_event = pd.DataFrame()

# Determine max length across lead columns (after filtering to season months)
max_len = 0
for lead_col in sum_f_indexed.columns:
    if not isinstance(lead_col, str) or '-' not in lead_col:
        continue
    days = int(lead_col.split('-')[0])
    flagged_idx = sum_f_indexed.index[sum_f_indexed[lead_col] != 0]
    time_event = (flagged_idx + pd.Timedelta(days=days)).normalize()
    time_event = pd.DatetimeIndex([d for d in time_event if d.month in allowed_months])
    max_len = max(max_len, len(time_event))

# Build padded columns
for lead_col in sum_f_indexed.columns:
    if not isinstance(lead_col, str) or '-' not in lead_col:
        continue
    days = int(lead_col.split('-')[0])
    flagged_idx = sum_f_indexed.index[sum_f_indexed[lead_col] != 0]
    time_event = (flagged_idx + pd.Timedelta(days=days)).normalize()
    time_event = pd.DatetimeIndex([d for d in time_event if d.month in allowed_months])

    padded = pd.Series(time_event.values, index=range(len(time_event))).reindex(range(max_len))
    dates_event[lead_col] = padded

# Add actual flood dates column (season months only)
actual_flood_dates_sorted = sorted(pd.to_datetime(list(actual_flood_dates)))
dates_event['Actual Flood Date'] = pd.Series(actual_flood_dates_sorted, index=range(len(actual_flood_dates_sorted))).reindex(range(max_len))

print("\n" + "="*100)
print("PREDICTED DATES BY LEAD DAY vs ACTUAL FLOOD DATES (All Years)")
print("="*100)
print(dates_event.to_string())

# Save detailed dates
out_dates = CONFIG['base_dir'] / 'event_dates_by_lead_all_years.csv'
dates_event.to_csv(out_dates, index=False)
print(f"\nSaved event dates to: {out_dates}")


EVENT-BASED EVALUATION METRICS (All Years) BY LEAD DAY + OVERALL
       Lead Day  TP (Hits)  FN (Misses)  FP (False Alarms)  TN (Correct Negatives)  Precision   Recall  F1 Score      FAR
          1-day          6            5                 29                     204   0.171429 0.545455  0.260870 0.828571
          2-day          5            6                 41                     192   0.108696 0.454545  0.175439 0.891304
          3-day          5            6                 48                     185   0.094340 0.454545  0.156250 0.905660
          4-day          8            3                 46                     187   0.148148 0.727273  0.246154 0.851852
          5-day          6            5                 45                     188   0.117647 0.545455  0.193548 0.882353
          6-day          2            9                 42                     191   0.045455 0.181818  0.072727 0.954545
          7-day          3            8                 44                     1

In [16]:
# Season-wise overall metrics (union-of-leads) for both models
# USING SPELL-BASED EVALUATION LOGIC
from pathlib import Path
import pandas as pd
import numpy as np

# Model directories
model_dirs = {
    "MONSOON": {
        "Logistic_Regression": Path(r"D:\Masters\data\logistic regression\predictions\logistic_regression\Logistic_Regression_t0.891\MONSOON"),
        "Random_Forest": Path(r"D:\Masters\data\logistic regression\predictions\random_forest\Random_Forest_t0.410\MONSOON"),
    },
    "PREMONSOON": {
        "Logistic_Regression": Path(r"D:\Masters\data\logistic regression\predictions\logistic_regression\Logistic_Regression_t0.887\PREMONSOON"),
        "Random_Forest": Path(r"D:\Masters\data\logistic regression\predictions\random_forest\Random_Forest_t0.400\PREMONSOON"),
    },
}

years = [2017, 2022, 2023, 2024]
obs_excel = Path("..") / "LSTM" / "PREMONSOON-janjul201724.xlsx"

# OUTPUT: Save to combined folder
combined_root = Path(r"D:\Masters\data\logistic regression\predictions\combined")
combined_root.mkdir(parents=True, exist_ok=True)
print(f"✓ Combined output folder: {combined_root}\n")

def season_months(season: str):
    s = season.upper()
    if s == "PREMONSOON":
        return 3, 5
    return 6, 7

def build_all_dates(years, start_month, end_month):
    alld = []
    for y in years:
        start = pd.Timestamp(y, start_month, 1)
        end = (pd.Timestamp(y, end_month, 1) + pd.offsets.MonthEnd(1)).normalize()
        rng = pd.date_range(start, end, freq='D')
        alld.extend(rng)
    return pd.DatetimeIndex(sorted(pd.to_datetime(alld).unique()))

def load_flood_spells(model_dir: Path, years: list[int]):
    """Load flood spell periods from flood_time_periods_{year}.csv files"""
    all_spells = []
    for y in years:
        fp = model_dir / f"flood_time_periods_{y}.csv"
        if not fp.exists():
            continue
        df = pd.read_csv(fp)
        df['Year'] = y
        all_spells.append(df)
    
    if not all_spells:
        return pd.DataFrame()
    
    return pd.concat(all_spells, ignore_index=True)

def build_spell_windows(spells_df):
    """Build spell window dictionaries from flood spells dataframe"""
    spell_windows = {}
    day_to_spells = {}
    
    for idx, row in spells_df.iterrows():
        spell_id = f"{row['Year']}_S{int(row['Spell_Number'])}"
        spell_start = pd.to_datetime(row['Spell_Start']).normalize().date()
        spell_end = pd.to_datetime(row['Spell_End']).normalize().date()
        
        # Hit window: [Spell_Start - 2 days, Spell_End]
        hit_window_start = (pd.to_datetime(spell_start) - pd.Timedelta(days=2)).date()
        hit_window_end = spell_end
        
        spell_windows[spell_id] = {
            'spell_start': spell_start,
            'spell_end': spell_end,
            'actual_flood_start': spell_start,
            'actual_flood_end': spell_end,
            'hit_window_start': hit_window_start,
            'hit_window_end': hit_window_end,
            'tol_window_start': hit_window_start,
            'tol_window_end': spell_end,
        }
        
        # Map each day to spell info
        for d in pd.date_range(hit_window_start, hit_window_end, freq='D'):
            if d.date() not in day_to_spells:
                day_to_spells[d.date()] = []
            day_to_spells[d.date()].append({
                'spell_id': spell_id,
                'in_hit_window': hit_window_start <= d.date() <= hit_window_end,
                'in_actual_flood': spell_start <= d.date() <= spell_end
            })
    
    return spell_windows, day_to_spells

def extract_pred_from_sumf(model_dir: Path, years: list[int], allowed_months: list[int], all_dates):
    pred_by_lead = {}
    for y in years:
        fp = model_dir / f"sum_f_{y}.csv"
        if not fp.exists():
            continue
        df = pd.read_csv(fp, index_col=0)
        idx = pd.to_datetime(df.index, errors='coerce')
        df.index = idx
        df = df[~df.index.isna()]
        lead_cols = [c for c in df.columns if isinstance(c, str) and "-day" in c]
        for c in lead_cols:
            try:
                days = int(str(c).split('-')[0])
            except:
                continue
            flagged_idx = df.index[df[c].fillna(0) != 0]
            verif_dates = (flagged_idx + pd.Timedelta(days=days)).normalize()
            verif_dates = verif_dates[verif_dates.month.isin(allowed_months)]
            verif_dates = verif_dates[verif_dates.isin(all_dates)]
            s = set(pd.to_datetime(verif_dates).date)
            if c not in pred_by_lead:
                pred_by_lead[c] = s
            else:
                pred_by_lead[c] |= s
    return pred_by_lead

def compute_confusion_spell_based(all_dates, spell_windows, day_to_spells, pred_set):
    """
    Compute confusion matrix using spell-based logic:
    - Hit window = [Spell_Start-2d, Spell_End]: predictions here = TP
    - Actual flood = [Spell_Start, Spell_End]: NO predictions here = FN (miss)
    - Early warning = [Spell_Start-2d, Spell_Start-1]: NOT counted as miss if empty
    - FP = predictions outside ALL tolerance windows
    """
    TP, FN, FP, TN = 0, 0, 0, 0
    
    # Build set of all dates in any tolerance window
    all_tolerance_dates = set()
    for window in spell_windows.values():
        for d in pd.date_range(window['tol_window_start'], window['tol_window_end'], freq='D'):
            all_tolerance_dates.add(d.date())
    
    for d in all_dates:
        d_date = d.date()
        is_predicted = d_date in pred_set
        
        # Check if this day is in any spell's window
        spell_info = day_to_spells.get(d_date, [])
        
        if spell_info:
            # This day is in hit window of at least one spell
            in_hit_window = any(s['in_hit_window'] for s in spell_info)
            in_actual_flood = any(s['in_actual_flood'] for s in spell_info)
            
            if in_hit_window:
                if is_predicted:
                    TP += 1
                elif in_actual_flood:
                    # Miss: in ACTUAL flood period with NO prediction
                    FN += 1
                # else: in early warning window with no prediction = NOT a miss
        else:
            # Day is NOT in any spell window
            if is_predicted:
                FP += 1  # False alarm
            else:
                TN += 1  # Correct non-prediction
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else np.nan
    recall = TP / (TP + FN) if (TP + FN) > 0 else np.nan
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 and not np.isnan(precision) and not np.isnan(recall) else np.nan
    far = FP / (TP + FP) if (TP + FP) > 0 else np.nan
    
    return TP, FN, FP, TN, precision, recall, f1, far

# Load obs once
obs_df_full = pd.read_excel(obs_excel)
obs_df_full['Date'] = pd.to_datetime(obs_df_full['Date Time']).dt.normalize()

for season, models in model_dirs.items():
    print(f"\n===== {season} =====")
    sm, em = season_months(season)
    allowed_months = list(range(sm, em + 1))
    
    # Combined output dir for this season
    out_dir = combined_root / season
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"Saving to: {out_dir}")

    all_dates = build_all_dates(years, sm, em)
    
    results_rows = []
    for model_name, dir_path in models.items():
        if not dir_path.exists():
            print(f"  ❌ {model_name}: not found")
            continue
        
        # Load flood spells for this model/season
        spells_df = load_flood_spells(dir_path, years)
        if spells_df.empty:
            print(f"  ❌ {model_name}: no flood spell data found")
            continue
        
        spell_windows, day_to_spells = build_spell_windows(spells_df)
        print(f"  → Loaded {len(spell_windows)} flood spells for spell-based evaluation")
        
        # Extract predictions
        pred_by_lead = extract_pred_from_sumf(dir_path, years, allowed_months, all_dates)
        if not pred_by_lead:
            print(f"  ❌ {model_name}: no predictions extracted")
            continue
        
        # Union-of-leads
        predicted_union = set()
        for s in pred_by_lead.values():
            predicted_union |= s

        # Compute metrics using SPELL-BASED LOGIC
        TP, FN, FP, TN, P, R, F1, FAR = compute_confusion_spell_based(
            all_dates, spell_windows, day_to_spells, predicted_union
        )

        # Save overall (union)
        overall_csv = out_dir / f"{model_name.replace(' ', '_')}_overall.csv"
        pd.DataFrame([{
            'Model': model_name, 
            'TP': TP, 'FN': FN, 'FP': FP, 'TN': TN, 
            'Precision': P, 'Recall': R, 'F1 Score': F1, 'FAR': FAR
        }]).to_csv(overall_csv, index=False)
        
        # Save per-lead (also using spell-based logic)
        per_lead_rows = []
        for lead, preds in pred_by_lead.items():
            t, f, fp, tn, p, r, f1, far = compute_confusion_spell_based(
                all_dates, spell_windows, day_to_spells, preds
            )
            per_lead_rows.append({
                'Lead Day': lead, 
                'TP': t, 'FN': f, 'FP': fp, 'TN': tn, 
                'Precision': p, 'Recall': r, 'F1 Score': f1, 'FAR': far
            })
        per_lead_csv = out_dir / f"{model_name.replace(' ', '_')}_per_lead.csv"
        pd.DataFrame(per_lead_rows).sort_values('Lead Day').to_csv(per_lead_csv, index=False)
        
        print(f"  ✓ {model_name}: saved overall & per-lead (spell-based metrics)")
        results_rows.append({
            'Model': model_name, 
            'TP': TP, 'FN': FN, 'FP': FP, 'TN': TN, 
            'Precision': round(P, 3), 'Recall': round(R, 3), 
            'F1': round(F1, 3), 'FAR': round(FAR, 3)
        })

    # Season comparison
    if results_rows:
        comp = pd.DataFrame(results_rows)
        comp_csv = out_dir / "overall_comparison.csv"
        comp.to_csv(comp_csv, index=False)
        print(f"\nSeason comparison (SPELL-BASED METRICS):")
        print(f"Hit window = [Spell_Start-2d, Spell_End]: predictions here = TP")
        print(f"Actual flood = [Spell_Start, Spell_End]: NO predictions here = FN")
        print(f"Early warning = [Spell_Start-2d, Spell_Start-1]: NOT counted as miss if empty")
        print(comp.to_string(index=False))
        print(f"Saved to: {comp_csv}")

print(f"\n✓✓✓ All outputs in combined folder: {combined_root}")


✓ Combined output folder: D:\Masters\data\logistic regression\predictions\combined


===== MONSOON =====
Saving to: D:\Masters\data\logistic regression\predictions\combined\MONSOON
  → Loaded 6 flood spells for spell-based evaluation
  ✓ Logistic_Regression: saved overall & per-lead (spell-based metrics)
  → Loaded 6 flood spells for spell-based evaluation
  ✓ Random_Forest: saved overall & per-lead (spell-based metrics)

Season comparison (SPELL-BASED METRICS):
Hit window = [Spell_Start-2d, Spell_End]: predictions here = TP
Actual flood = [Spell_Start, Spell_End]: NO predictions here = FN
Early warning = [Spell_Start-2d, Spell_Start-1]: NOT counted as miss if empty
              Model  TP  FN  FP  TN  Precision  Recall    F1   FAR
Logistic_Regression  13   3  46 178      0.220   0.812 0.347 0.780
      Random_Forest  20   0 154  70      0.115   1.000 0.206 0.885
Saved to: D:\Masters\data\logistic regression\predictions\combined\MONSOON\overall_comparison.csv

===== PREMONSOON =====
Sa